# **10.1 Introduction To Equivariance and Equivariant Modeling**

Invariance : 입력은 바꿔도 출력은 그대로 (분자를 90도 회전시켜도 에너지는 동일)

Equivariance : 입력을 바꾼만큼 똑같이 변함 (분자를 90도 회전시키면 힘의 방향도 90도 회전)

1. 구조적 정보 보존

2. 대칭성과 불변성 처리

3. 일반화 능력 향상

4. 그래프 구조 데이터 효율적 처

In [1]:
import numpy as np
np.random.seed(42) # seed for reproducibility

R_i = np.random.rand(12, 3) # 12 atoms with xyz coordinates
N = R_i.shape[0] # number of atoms
X_i = np.zeros((N, 2)) # feature vectors for the atoms with shape (N, 2)
X_i[:4, 0] = 1
X_i[4:, 1] = 1

In [2]:
def hidden_model(r: np.ndarray, x: np.ndarray, w1: np.ndarray, w2: np.ndarray, b1: np.ndarray, b2: float) -> np.ndarray:
    r"""Computes the output of a 1-hidden layer neural network model.

    Parameters
    ----------
        r : np.ndarray
            Input array for position values.
            Shape: (num_atoms, num_positions)
        x : np.ndarray
            Input array for features.
            Shape: (num_atoms, num_features)
        w1 : np.ndarray
            Weight matrix for the first layer.
            Shape: (num_positions + num_features, hidden_size)
        w2 : np.ndarray
            Weight matrix for the second layer.
            Shape: (hidden_size, output_size)
        b1 : np.ndarray
            Bias vector for the first layer.
            Shape: (hidden_size,)
        b2 : float
            Bias value for the second layer.

    Returns
    -------
        float
            Predicted energy of the molecule
    """
    i = np.concatenate((r, x), axis=1).flatten()  # Stack inputs into one large input
    v = np.tanh(i @ w1 + b1)  # Apply activation function to first layer
    v = v @ w2 + b2  # Multiply with weights and add bias for the second layer
    return v


# Initialize weights for a network with hidden size is 16
w1 = np.random.normal(size=(N * 5, 16))  # 3(#positions) + 2(#features) = 5
b1 = np.random.normal(size=(16,))
w2 = np.random.normal(size=(16,))
b2 = np.random.normal()

In [3]:
import scipy.spatial.transform as transform

rotate = transform.Rotation.from_euler("x", 60, degrees=True) # Rotate around x axis by 60 degrees

permuted_R_i = np.copy(R_i)
permuted_R_i[0], permuted_R_i[1] = R_i[1], R_i[0] # Swap the rows of R_i

print("without change:", hidden_model(R_i, X_i, w1, w2, b1, b2))
print("after permutation:", hidden_model(permuted_R_i, X_i, w1, w2, b1, b2))
print("after translation:", hidden_model(R_i + np.array([3, 3, 3]), X_i, w1, w2, b1, b2))
print("after rotation:", hidden_model(rotate.apply(R_i), X_i, w1, w2, b1, b2))

without change: 4.0914311641808805
after permutation: 2.235218761280032
after translation: -0.15670614466272625
after rotation: 0.5155425905215233


In [4]:
def hidden_model_perm(r: np.ndarray, x: np.ndarray, w1: np.ndarray, w2: np.ndarray, b1: np.ndarray, b2: float) -> np.ndarray:
    r"""Computes the output of a 1-hidden layer neural network model with permutation invariance.

    Parameters
    ----------
        r : np.ndarray
            Input array for position values.
            Shape: (num_atoms, num_positions)
        x : np.ndarray
            Input array for features.
            Shape: (num_atoms, num_features)
        w1 : np.ndarray
            Weight matrix for the first layer.
            Shape: (num_positions + num_features, hidden_size)
        w2 : np.ndarray
            Weight matrix for the second layer.
            Shape: (hidden_size, output_size)
        b1 : np.ndarray
            Bias vector for the first layer.
            Shape: (hidden_size,)
        b2 : float
            Bias value for the second layer.

    Returns
    -------
        float
            Predicted energy of the molecule
    """
    i = np.concatenate((r, x), axis=1)  # Stack inputs into one large input
    v = np.tanh(i @ w1 + b1)  # Apply activation function to first layer
    v = np.sum(v, axis=0) # Reduce the output by summing across the axis which gives permutational invariance
    v = v @ w2 + b2  # Multiply with weights and add bias for the second layer
    return v

# Initialize weights
w1 = np.random.normal(size=(5, 16))
b1 = np.random.normal(size=(16,))
w2 = np.random.normal(size=(16,))
b2 = np.random.normal()

In [5]:
print("without change:", hidden_model_perm(R_i, X_i, w1, w2, b1, b2))
print("after permutation:", hidden_model_perm(permuted_R_i, X_i, w1, w2, b1, b2))
print("after translation:", hidden_model_perm(R_i + np.array([3, 3, 3]), X_i, w1, w2, b1, b2))
print("after rotation:", hidden_model_perm(rotate.apply(R_i), X_i, w1, w2, b1, b2))

without change: 63.42865014451651
after permutation: 63.42865014451651
after translation: 100.6256277697755
after rotation: 66.30189546860116


일반모델 -> 모든 원자의 좌표와 특징을 하나로 길게 펼쳐서 입력 -> 동일한 분자임에도  순서바꿈(Permutation), 옆으로 옮김(Translation), 회전(Rotation) 하면 예측값 바뀜


Permutational Invariance 해결

1. 모든 원자에 대해 동일한 가중치 적용해 원자의 특징 먼저 계산

2. 그 결과값들을 모두 더함

-> 원자의 순서가 바뀌어도 합계는 변하지 않아 불변성 확보


But

Permutation값은 동일하지만 이동과 회전에 대해서는 달라짐

입력값으로 절대좌표(x,y,z)를 사용하고 있기 때문에

In [6]:
def hidden_model_permute_translate(r: np.ndarray, x: np.ndarray, w1: np.ndarray, w2: np.ndarray, b1: np.ndarray, b2: float) -> np.ndarray:
    r"""Computes the output of a 1-hidden layer neural network model with permutation and translation invariance.

    Parameters
    ----------
        r : np.ndarray
            Input array for position values.
            Shape: (num_atoms, num_positions)
        x : np.ndarray
            Input array for features.
            Shape: (num_atoms, num_features)
        w1 : np.ndarray
            Weight matrix for the first layer.
            Shape: (num_positions + num_features, hidden_size)
        w2 : np.ndarray
            Weight matrix for the second layer.
            Shape: (hidden_size, output_size)
        b1 : np.ndarray
            Bias vector for the first layer.
            Shape: (hidden_size,)
        b2 : float
            Bias value for the second layer.

    Returns
    -------
        float
            Predicted energy of the molecule
    """
    d = r - r[:, np.newaxis] # Compute pairwise distances using broadcasting

    # Stack inputs into one large input of N x N x 5
    # Concatenate doesn't broadcast, so we manually broadcast the Nx2 x matrix
    # into N x N x 2
    i = np.concatenate((d, np.broadcast_to(x, (d.shape[:-1] + x.shape[-1:]))), axis=-1)
    v = np.tanh(i @ w1 + b1)  # Apply activation function to first layer

    v = np.sum(v, axis=(0, 1)) # Reduce the output over both axes by summing
    v = v @ w2 + b2  # Multiply with weights and add bias for the second layer
    return v

In [7]:
print("without change:", hidden_model_permute_translate(R_i, X_i, w1, w2, b1, b2))
print("after permutation:", hidden_model_permute_translate(permuted_R_i, X_i, w1, w2, b1, b2))
print("after translation:", hidden_model_permute_translate(R_i + np.array([3, 3, 3]), X_i, w1, w2, b1, b2))
print("after rotation:", hidden_model_permute_translate(rotate.apply(R_i), X_i, w1, w2, b1, b2))

without change: 238.78854708962047
after permutation: 238.78854708962035
after translation: 238.7885470896204
after rotation: 262.5920439748347


Translational Invariance

절대좌표를 쓰면 이동해도 값이 변함 -> 상대적 위치 벡터 계산

벡터는 회전하면 방향이 바뀌어 rotation은 여전치 다름

In [8]:
def hidden_model_permute_translate_rotate(r: np.ndarray, x: np.ndarray, w1: np.ndarray, w2: np.ndarray, b1: np.ndarray, b2: float) -> np.ndarray:
    r"""Computes the output of a 1-hidden layer neural network model with permutation, translation, and rotation invariance.

    Parameters
    ----------
        r : np.ndarray
            Input array for position values.
            Shape: (num_atoms, num_positions)
        x : np.ndarray
            Input array for features.
            Shape: (num_atoms, num_features)
        w1 : np.ndarray
            Weight matrix for the first layer.
            Shape: (num_positions, hidden_size)
        w2 : np.ndarray
            Weight matrix for the second layer.
            Shape: (hidden_size, output_size)
        b1 : np.ndarray
            Bias vector for the first layer.
            Shape: (hidden_size,)
        b2 : float
            Bias value for the second layer.

    Returns
    -------
        float
            Predicted energy of the molecule
    """
    # Compute pairwise distances using broadcasting
    d = r - r[:, np.newaxis]
    # Compute squared distances
    d2 = np.sum(d**2, axis=-1, keepdims=True)

    # Stack inputs into one large input of N x N x 3
    # Concatenate doesn't broadcast, so we manually broadcast the Nx2 x matrix
    # into N x N x 2
    i = np.concatenate((d2, np.broadcast_to(x, (d2.shape[:-1] + x.shape[-1:]))), axis=-1)
    v = np.tanh(i @ w1 + b1)  # Apply activation function to first layer

    # Reduce the output over both axes by summing
    v = np.sum(v, axis=(0, 1))

    v = v @ w2 + b2  # Multiply with weights and add bias for the second layer
    return v

# Initialize weights
w1 = np.random.normal(size=(3, 16))
b1 = np.random.normal(size=(16,))
w2 = np.random.normal(size=(16,))
b2 = np.random.normal()

In [11]:
print("without change:", hidden_model_permute_translate_rotate(R_i, X_i, w1, w2, b1, b2))
print("after permutation:", hidden_model_permute_translate_rotate(permuted_R_i, X_i, w1, w2, b1, b2))
print("after translation:", hidden_model_permute_translate_rotate(R_i + np.array([3, 3, 3]), X_i, w1, w2, b1, b2))
print("after rotation:", hidden_model_permute_translate_rotate(rotate.apply(R_i), X_i, w1, w2, b1, b2))

without change: 654.3040623148685
after permutation: 654.3040623148685
after translation: 654.3040623148685
after rotation: 654.3040623148685


Rotational Invariance

벡터는 회전하면 숫자 바뀜, 에너지는 스칼라 값이므로 회전해도 일정

상대적 위치 벡터의 제공거리(d^2) 이용

벡터의 길이는 분자를 rotation해도 변하지 않음 -> rotation후에도 일치

💡Insight

절대 좌표로 순서 불변성 확

상대 좌표로 이동 불변성 확보

거리 행렬을 통해 회전 불변성 구현